In [ ]:
from oganesson.descriptors import BACD, SymmetryFunctions, DScribeSOAP, DScribeSineMatrix
from oganesson.ogstructure import OgStructure


bacd = BACD(OgStructure(file_name='examples/structures/mp-541001_LiInI4.cif'))
bacd.describe()

sf = DScribeSineMatrix(OgStructure(file_name='examples/structures/mp-541001_LiInI4.cif'))
sf.describe()


In [ ]:
sf.dscribe.create()

In [ ]:
from dscribe.descriptors import SOAP
from ase.build import molecule

species = ["H", "C", "O", "N"]
rcut = 6.0
nmax = 8
lmax = 6

# Setting up the SOAP descriptor
soap = SOAP(
    species=species,
    periodic=False,
    rcut=rcut,
    nmax=nmax,
    lmax=lmax,
)
water = molecule("H2O")

# Create SOAP output for the system
soap_water = soap.create(water, positions=[0])


In [ ]:
from oganesson.descriptors import BACD, Describe
# from oganesson.descriptors import DScribeSOAP, DscribeACSF
from oganesson.ogstructure import OgStructure

bacd = BACD(OgStructure(file_name='examples/structures/Li3PO4_mp-13725.cif'))
print(bacd.describe())
# soap = DScribeSOAP(OgStructure(file_name='examples/structures/Li3PO4_mp-13725.cif'))
# acsf = DscribeACSF(OgStructure(file_name='examples/structures/Li3PO4_mp-13725.cif'))

print(Describe.describe(OgStructure(file_name='examples/structures/Li3PO4_mp-13725.cif'),BACD))

In [ ]:
from oganesson.ogstructure import OgStructure

og=OgStructure(file_name='examples/structures/MoS2.vasp')
og.add_atom_to_surface('Li').structure.to('MoS2_Li.vasp','poscar')

In [ ]:
from oganesson.ogstructure import OgStructure
og = OgStructure(file_name='examples/structures/Li3PO4_mp-13725.cif')
og.generate_neb('Li', r=3)


In [ ]:
## Generation of machine learning datasets

In [ ]:
import requests
from oganesson.descriptors import Describe, BACD
from oganesson.ogstructure import OgStructure
from pymatgen.core.structure import Structure
from joltml import Experiment, Xgboost, Sklearn
import pandas as pd

headers = {
    'accept': 'application/json',
    'X-API-KEY': 'iGRUQOIQAcPMw00QWQKIEegfhF8O7Gmm'
}
materials_summary = requests.get('https://api.materialsproject.org/materials/summary/?formula=ABO3&deprecated=false&_per_page=1000&_skip=0&_limit=1000&_all_fields=true&is_stable=true', headers=headers)
materials_summary = materials_summary.json()['data']
datasets = {'material_ids':[],'structures':[],'bacd':[],'formation_energy_per_atom':[]}
for material in materials_summary:
    structure = OgStructure(Structure.from_dict(material['structure']))
    datasets['material_ids'] += [material['material_id']]
    datasets['structures'] += [structure]
    datasets['bacd'] += [Describe.describe(structure,descriptor=BACD)]
    datasets['formation_energy_per_atom'] += [material['formation_energy_per_atom']]


In [ ]:

import numpy as np
training_set = pd.DataFrame(datasets['bacd'])
experiment = Experiment(training_set.iloc[:800])
y = experiment.add_models([Xgboost()]).regression(targets=pd.DataFrame(datasets['formation_energy_per_atom'][:800]), splits=[0.8,0.2]).predict(training_set.iloc[:800])



In [ ]:

# Test sensitivity of descriptors

# Test translational and rotational symmetry of descriptors


In [ ]:
from oganesson.descriptors import BACD
from oganesson.ogstructure import OgStructure
bacd = BACD(OgStructure(file_name='examples/structures/Li3PO4_mp-13725.cif'))
print(bacd.is_invariant())

## Performing AIMD simulations

Here we apply a lithium-rich material, Li3PO4, to a temperature of 1000 K, and then calculate the diffusivities of the three elements.

In [1]:
from oganesson.ogstructure import OgStructure
structure = OgStructure(file_name='examples/structures/LGPS_ChemMater_2018_30_4995_Opt.cif')
structure.simulate(temperature=1000,steps=1000,loginterval=1)
coeffs = structure.calculate_diffusivity()
print('Diffusion coefficients:', coeffs)


GPAW is not installed, and therefore you cannot use the ROSA descriptors.
DScribe is not installed, and therefore you cannot use the DScribe descriptors.
Plotting MSD using 1002 images
Diffusion coefficients: ([8.929355972110487e-05, -6.759250973865885e-07, 4.638191265231533e-06, 1.2941813343037413e-05], [0.0, 0.0, 0.0, 1.6940658945086007e-21])


<Figure size 1500x1000 with 0 Axes>